In [20]:
import math
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Set the path to the dataset
train_dir = 'C:\\Users\\User\\Downloads\\Archive\\train'
test_dir = 'C:\\Users\\User\\Downloads\\Archive\\test'
val_dir = 'C:\\Users\\User\\Downloads\\Archive\\val'


In [10]:
# Define hyperparameters
input_shape = (224, 224, 3)
batch_size = 32
epochs = 10

In [11]:
# Create a CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))  # Output layer for binary classification with 2 classes

In [12]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
# Create data generators with augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(input_shape[0], input_shape[1]),
                                                    batch_size=batch_size,
                                                    class_mode='sparse')

validation_generator = test_datagen.flow_from_directory(val_dir,
                                                        target_size=(input_shape[0], input_shape[1]),
                                                        batch_size=batch_size,
                                                        class_mode='sparse')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [23]:
# Calculate steps_per_epoch for training data
steps_per_epoch_train = math.ceil(len(train_generator) / batch_size)

In [24]:
# Calculate steps_per_epoch for validation data
steps_per_epoch_validation = math.ceil(len(validation_generator) / batch_size)

In [25]:
# Train the model
try:
    history = model.fit(train_generator,
                        steps_per_epoch=steps_per_epoch_train,
                        epochs=epochs,
                        validation_data=validation_generator,
                        validation_steps=steps_per_epoch_validation)
except Exception as e:
    print("An error occurred during training:", e)

Epoch 1/10
6/6 [==============================] - 37s 7s/step - loss: 0.2559 - accuracy: 0.8958 - val_loss: 0.4435 - val_accuracy: 0.8125
Epoch 2/10
6/6 [==============================] - 15s 2s/step - loss: 0.1729 - accuracy: 0.9375 - val_loss: 0.5521 - val_accuracy: 0.8750
Epoch 3/10
6/6 [==============================] - 16s 3s/step - loss: 0.2099 - accuracy: 0.9271 - val_loss: 0.9659 - val_accuracy: 0.6875
Epoch 4/10
6/6 [==============================] - 14s 2s/step - loss: 0.1446 - accuracy: 0.9323 - val_loss: 0.8015 - val_accuracy: 0.8125
Epoch 5/10
6/6 [==============================] - 14s 2s/step - loss: 0.1936 - accuracy: 0.9115 - val_loss: 1.1295 - val_accuracy: 0.6250
Epoch 6/10
6/6 [==============================] - 16s 2s/step - loss: 0.1401 - accuracy: 0.9479 - val_loss: 0.8256 - val_accuracy: 0.6875
Epoch 7/10
6/6 [==============================] - 13s 2s/step - loss: 0.2424 - accuracy: 0.9010 - val_loss: 0.5187 - val_accuracy: 0.8750
Epoch 8/10
6/6 [==================

In [26]:
# Evaluate the model on the test set
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(input_shape[0], input_shape[1]),
                                                  batch_size=batch_size,
                                                  class_mode='sparse')

test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test accuracy: {test_acc}')

Found 624 images belonging to 2 classes.
19/19 [==============================] - 42s 2s/step - loss: 0.3300 - accuracy: 0.8388
Test accuracy: 0.8388158082962036


In [27]:
# Save the model in Keras native format
model.save('image_classification_model.keras')